In [6]:
# %pip install torch
# %pip install torchvision
# %pip install albumentations
# %pip install onnx
# %pip install opencv-python

In [2]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import time
from torchvision.models import resnet50, ResNet50_Weights
import openvino.torch

# Load pretrained ResNet50 model
r50 = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

# Load the same model but scripted with torch.jit.script
# r50_scripted = torch.jit.script(r50)
r50_scripted = torch.compile(r50, backend='openvino')

# Set the models to evaluation mode
r50.eval()
r50_scripted.eval()

# Define image preprocessing pipeline
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to perform inference with a model on a single image
def inference(model, image_path):
    image = Image.open(image_path)
    image_tensor = preprocess(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        output = model(image_tensor)
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    _, predicted_class = torch.max(probabilities, 0)
    return predicted_class.item()

# Function to test accuracy on a set of images
def test_accuracy(model):
    correct = 0
    total = 0
    with torch.no_grad():
        for image_path, label in test_dataset:
            predicted_class = inference(model, image_path)
            if predicted_class == label:
                correct += 1
            total += 1
    accuracy = correct / total
    return accuracy

# Paths to sample images and corresponding labels
test_dataset = [("test_images/analog_clock_409.jpg", 409), ("test_images/burrito_965.jpg", 965), ("test_images/conch_112.jpg", 112), 
("test_images/peir_718.jpg", 718), ("test_images/starfish_327.jpg", 327), ("test_images/zebra_340.jpg", 340), ]

# Test accuracy of both models
accuracy_r50 = test_accuracy(r50)
accuracy_r50_scripted = test_accuracy(r50_scripted)

print("Accuracy of ResNet50 model:", accuracy_r50)
print("Accuracy of Scripted ResNet50 model:", accuracy_r50_scripted)

# Test inference time of both models
start_time = time.time()
inference(r50, "test_images/analog_clock_409.jpg")
end_time = time.time()
inference_time_r50 = end_time - start_time

start_time = time.time()
inference(r50_scripted, "test_images/analog_clock_409.jpg")
end_time = time.time()
inference_time_r50_scripted = end_time - start_time

print("Inference time of ResNet50 model:", inference_time_r50)
print("Inference time of Scripted ResNet50 model:", inference_time_r50_scripted)

c:\Users\wipar\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Windows not yet supported for torch.compile